# MSA Analysis - General Model for Machine Learning Prediction

- Use this notebook to pick a model that will be used for machine learning prediction within an MSA. 
- The code needs to be updated to refine and pick variable names
- Plotting is not finished or refined 

In [2]:
#import functions
import os
import sys

sys.path.append(
    os.path.join(os.path.abspath(os.path.join(os.path.curdir, os.path.pardir)))
)

#censusdis
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt

from typing import Optional

import censusdis.data as ced
import censusdis.maps as cem
import censusdis.values as cev
import censusdis.geography as cgeo
from censusdis.states import STATE_MA
from censusdis import states
from censusdis.maps import ShapeReader, plot_us_boundary
import censusdis.maps as cmap


# Make sure it is there.
from censusdis.values import ALL_SPECIAL_VALUES

# _______________________________________________________________________

#standard packages
import pandas as pd
import numpy as np
import math
from math import pi, sqrt
import matplotlib.pyplot as plt
# import pygwalker as pyg

# import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import datetime
import time
# from tqdm import tqdm, trange

#gis packages
import osmnx as ox
import logging as lg
ox.settings.log_console=True #use cache to avoid overloading the server
# ox.settings.memory_cache=True #use cache to avoid overloading the server
ox.settings.memory = 4294967296 #set memory cache to 4GB

from shapely.geometry import Point
import folium
import networkx as nx
## future libaries
# import contextily as cx
# import fiona
# from pandana.loaders import osm
# import momepy
# import missingno as msno
# from us import states
# import imageio

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
ox.__version__



pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

In [3]:
# summaryfiles_test = pd.read_csv('C:/Users/jerem/OneDrive/Documents/Git Projects/MeridianXYZ/data/census/acs/summary files/Tracts_Block_Groups_Only/Massachusetts_Tracts_Block_Groups_Only/e20215ma0001000.txt')
# summaryfiles_test.head()

In [4]:
# df1 = pd.read_csv('C:/Users/jerem/OneDrive/Documents/Git Projects/MeridianXYZ/data/census/acs/summary files/Tracts_Block_Groups_Only/Massachusetts_Tracts_Block_Groups_Only/g20215ma.csv')
# df1.head()

In [5]:
# api_key = "781655d3b92f252d48d8acae0f5669dddf3cb9a4"

# 

# Choose variable list from csv file

In [6]:
#load csv and drop old index file
#choose model type - in this case "Gemeral Model"

acs5 = pd.read_csv('C:/Users/jerem/OneDrive/Documents/Git Projects/MeridianXYZ/data/census/ACS Variables.csv')
acs5 = acs5[(acs5['General Model'] == 1) & (acs5['ACS1'] != 1)].reset_index().drop(columns = ['index'])
acs5

,Topic,Category,Subcategory,Details,Code,Table Details,General Model,ACS1,Census
0,Age and Sex,Sex by Age,Age and Sex,NaN,B01001,‡†,1.0,NaN,NaN
1,Age and Sex,Median Age by Sex,Age and Sex,NaN,B01002,†,1.0,NaN,NaN
2,Age and Sex,Total Population,Age and Sex,NaN,B01003,NaN,1.0,NaN,NaN
3,Children,Population Under 18 Years by Age,Children,NaN,B09001,NaN,1.0,NaN,NaN
4,Children,Relationship by Household Type (Including Living Alone),Children,NaN,B09020,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
119,Public Assistance,Receipt of Food Stamps/SNAP by Presence of Children by Household Type for Households,Public Assistance,NaN,B22002,NaN,1.0,NaN,NaN
120,Public Assistance,Receipt of Food Stamps/SNAP by Poverty Status for Households,Public Assistance,NaN,B22003,NaN,1.0,NaN,NaN
121,Race and Hispanic Origin,Race,Details,NaN,B02001,NaN,1.0,NaN,NaN
122,Veterans and Military,Sex by Age by Veteran Status for the Civilian Population 18 Years and Over,Veterans and Military,NaN,B21001,‡†,1.0,NaN,NaN


In [7]:
# acs1 = pd.read_csv('C:/Users/jerem/OneDrive/Documents/Git Projects/MeridianXYZ/data/census/ACS Variables.csv')
# acs1 = acs1[(acs1['General Model'] == 1) & (acs1['ACS1'] == 1)].reset_index().drop(columns = ['index'])
# acs1

In [8]:
# extract single variable from group list
value = acs5.loc[1, 'Code']
value

group_acs5 = acs5['Code'].to_list()
print(len(group_acs5))
group_acs5[:3]

124


['B01001', 'B01002', 'B01003']

In [9]:
# # extract single variable from group list
# value = acs1.loc[1, 'Code']
# value

# group_acs1 = acs1['Code'].to_list()
# print(len(group_acs1))
# group_acs1

In [10]:
dataset = 'acs/acs5'
year = 2022

# get readable variable names ACS5

group_acs5_names = []
group_acs5_updated = []

for i in range(len(group_acs5)):
    try:
        #test which group is going through the loop
        # print(i)
        # print(f'Group: {group[i]}')
    
        #get the first group variable
        group_variables = ced.variables.group_variables(dataset, year, group_acs5[i])
        group_variables = group_variables[0]
        # print(group_vaariables)
    
        #get the dict for each group
        get_group = ced.variables.get_group(dataset, year, group_acs5[i])
        group_info = get_group[group_variables]
        # print(group_info)
    
        #get readable name
        label = get_group[group_variables]['label'].replace(' --','').replace(':!!', '-').replace('!!','_').replace(' ','_').replace(':','')
        concept = get_group[group_variables]['concept'].replace(' ','_')
        name = concept + " - " + label
        # print(name)
        # print(" ")
    
        group_acs5_names.append(name)
        group_acs5_updated.append(group_variables)
        
    except: 
        print(f'Group: {group_acs5[i]} FAILED')
        print(" ")
        continue
print(len(group_acs5_names))
print(len(group_acs5_updated))

group_acs5_updated
group_acs5_names

124
124


['Sex_by_Age - Estimate_Total',
 'Median_Age_by_Sex - Estimate_Median_age_Total',
 'Total_Population - Estimate_Total',
 'Population_Under_18_Years_by_Age - Estimate_Total',
 'Relationship_by_Household_Type_(Including_Living_Alone)_for_the_Population_65_Years_and_Over - Estimate_Total',
 'Women_15_to_50_Years_Who_Had_a_Birth_in_the_Past_12_Months_by_Age - Estimate_Total',
 'Aggregate_Travel_Time_to_Work_(in_Minutes)_of_Workers_by_Travel_Time_to_Work - Estimate_Aggregate_travel_time_to_work_(in_minutes)',
 'Travel_Time_to_Work - Estimate_Total',
 'Travel_Time_to_Work_for_Workplace_Geography - Estimate_Total',
 'Worker_Population_for_Workplace_Geography - Estimate_Total',
 'Median_Age_by_Means_of_Transportation_to_Work - Estimate_Median_age_Total',
 'Means_of_Transportation_to_Work - Estimate_Total',
 'Sex_by_Age_by_Disability_Status - Estimate_Total',
 'Sex_by_School_Enrollment_by_Type_of_School_by_Age_for_the_Population_3_Years_and_Over - Estimate_Total',
 'Sex_by_Age_by_Educational_At

In [13]:
# ced.variables.group_variables(dataset, year, 'B01001')

In [12]:
variable_and_names = pd.DataFrame(list(zip(group_acs5_updated, group_acs5_names)), columns=['variables', 'names'])
variable_and_names

,variables,names
0,B01001_001E,Sex_by_Age - Estimate_Total
1,B01002_001E,Median_Age_by_Sex - Estimate_Median_age_Total
2,B01003_001E,Total_Population - Estimate_Total
3,B09001_001E,Population_Under_18_Years_by_Age - Estimate_Total
4,B09020_001E,Relationship_by_Household_Type_(Including_Living_Alone)_for_the_Population_65_Years_and_Over - Estimate_Total
...,...,...
119,B22002_001E,Receipt_of_Food_Stamps/SNAP_in_the_Past_12_Months_by_Presence_of_Children_Under_18_Years_by_Household_Type_for_Households - Estimate_Total
120,B22003_001E,Receipt_of_Food_Stamps/SNAP_in_the_Past_12_Months_by_Poverty_Status_in_the_Past_12_Months_for_Households - Estimate_Total
121,B02001_001E,Race - Estimate_Total
122,B21001_001E,Sex_by_Age_by_Veteran_Status_for_the_Civilian_Population_18_Years_and_Over - Estimate_Total


In [ ]:
# dataset = 'acs/acs1'
# year = 2022

# # get human readable variable names ACS5

# group_acs1_names = []
# group_acs1_updated = []

# for i in range(len(group_acs1)):
#     try:
#         #test which group is going through the loop
#         # print(i)
#         # print(f'Group: {group[i]}')
    
#         #get the first group variable
#         group_variables = ced.variables.group_variables(dataset, year, group_acs1[i])
#         group_variables = group_variables[0]
#         # print(group_vaariables)
    
#         #get the dict for each group
#         get_group = ced.variables.get_group(dataset, year, group_acs1[i])
#         group_info = get_group[group_variables]
#         # print(group_info)
    
#         #get readable name
#         label = get_group[group_variables]['label'].replace(' --','').replace(':!!', '-').replace('!!','_').replace(' ','_').replace(':','')
#         concept = get_group[group_variables]['concept'].replace(' ','_')
#         name = concept + " - " + label
#         # print(name)
#         # print(" ")
    
#         group_acs1_names.append(name)
#         group_acs1_updated.append(group_variables)
        
#     except: 
#         print(f'Group: {group_acs1[i]} FAILED')
#         print(" ")
#         continue
# print(len(group_acs1_names))
# print(len(group_acs1_updated))

# group_acs1_updated
# group_acs1_names

In [ ]:
# help(ced.variables.group_tree)
# help(ced.variables)
# help(ced.variables.group_variables)

***
# Get Data from Censusdis

- combine the data from the ACS5 file to a CensusDIS shape file. 
- * problem is the data needs to be chunked when joined, need to find a better way to accomplish this
***

In [ ]:
dataset = 'acs/acs5'
year = 2022
state = STATE_MA

In [ ]:
reader = ShapeReader(year=year)

In [ ]:
variables = 'NAME'
counties_ma = ['017', '009', '021', '023', '025'] # ['Middlesex County, Massachusetts', 'Essex County, Massachusetts', 'Norfolk County, Massachusetts', 'Plymouth County, Massachusetts', 'Suffolk County, Massachusetts']

polygon_block_groups = ced.download(
    dataset,
    year,
    variables,
    state = state,
    county = counties_ma,
    tract = '*',
    block_group = "*",
    with_geometry = True,
    # remove_water = True,
    set_to_nan=ALL_SPECIAL_VALUES
)

polygon_block_groups

#### Check Tests

In [ ]:
def chunk_list(lst, chunk_size):
    """Yield successive chunk_size chunks from lst."""
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]


def process_data(data_chunk):
    """Process the data in some way."""
    processed = [item * 2 for item in data_chunk]  # Example processing
    return processed

# Example list
data = list(range(10))

# Break the list into chunks of size 10
chunk_size = 10
chunks = chunk_list(data, chunk_size)
chunks

# Process each chunk and collect the results
results = []
for chunk in chunks:
    processed_chunk = process_data(chunk)
    results.extend(processed_chunk)

data

results

In [ ]:
def process_data(data_chunk):
    polygon_block_groups = ced.download(
        dataset,
        year,
        data_chunk, #variables,
        state = state,
        county = counties_ma,
        tract = '*',
        block_group = "*",
        # with_geometry = True,
        # remove_water = True,
        set_to_nan=ALL_SPECIAL_VALUES
    )
    return polygon_block_groups


# Break the list into chunks of size 10
chunk_size = 40
chunks = chunk_list(group_updated, chunk_size)

# Process each chunk and collect the results
results = []
for chunk in chunks:
    processed_chunk = process_data(chunk)
    results.extend(processed_chunk)

results

#### Chunk Tests over

In [ ]:
# Variables
# TOTAL_POPULATION_VARIABLE = "B01003_001E"
variables_0 = 'NAME'
variables_1 = group_acs5_updated[:40]
variables_2 = group_acs5_updated[40:80]
variables_3 = group_acs5_updated[80:120]
variables_4 = group_acs5_updated[120:]


counties_ma = ['017', '009', '021', '023', '025'] # ['Middlesex County, Massachusetts', 'Essex County, Massachusetts', 'Norfolk County, Massachusetts', 'Plymouth County, Massachusetts', 'Suffolk County, Massachusetts']

In [ ]:
#0
polygon_block_groups_0 = ced.download(
    dataset,
    year,
    variables_0,
    state = state,
    county = counties_ma,
    tract = '*',
    block_group = "*",
    with_geometry = True,
    # remove_water = True,
    set_to_nan=ALL_SPECIAL_VALUES
)

#1
polygon_block_groups_1 = ced.download(
    dataset,
    year,
    variables_1,
    state = state,
    county = counties_ma,
    tract = '*',
    block_group = "*",
    with_geometry = True,
    # remove_water = True,
    set_to_nan=ALL_SPECIAL_VALUES
)

#2
polygon_block_groups_2 = ced.download(
    dataset,
    year,
    variables_2,
    state = state,
    county = counties_ma,
    tract = '*',
    block_group = "*",
    with_geometry = True,
    # remove_water = True,
    set_to_nan=ALL_SPECIAL_VALUES
)

#3
polygon_block_groups_3 = ced.download(
    dataset,
    year,
    variables_3,
    state = state,
    county = counties_ma,
    tract = '*',
    block_group = "*",
    with_geometry = True,
    # remove_water = True,
    set_to_nan=ALL_SPECIAL_VALUES
)

#4
polygon_block_groups_4 = ced.download(
    dataset,
    year,
    variables_4,
    state = state,
    county = counties_ma,
    tract = '*',
    block_group = "*",
    with_geometry = True,
    # remove_water = True,
    set_to_nan=ALL_SPECIAL_VALUES
)

In [ ]:
# polygon_block_groups_0
# polygon_block_groups_1
# polygon_block_groups_2
# polygon_block_groups_3
# polygon_block_groups_4

In [ ]:
polygon_block_groups = polygon_block_groups.dropna(subset=['geometry'])
polygon_block_groups_0 = polygon_block_groups_0.dropna(subset=['geometry']).drop(columns=['STATE','COUNTY','TRACT','geometry'])
polygon_block_groups_1 = polygon_block_groups_1.dropna(subset=['geometry']).drop(columns=['STATE','COUNTY','TRACT','geometry'])
polygon_block_groups_2 = polygon_block_groups_2.dropna(subset=['geometry']).drop(columns=['STATE','COUNTY','TRACT','geometry'])
polygon_block_groups_3 = polygon_block_groups_3.dropna(subset=['geometry']).drop(columns=['STATE','COUNTY','TRACT','geometry'])
polygon_block_groups_4 = polygon_block_groups_4.dropna(subset=['geometry']).drop(columns=['STATE','COUNTY','TRACT','geometry'])

result_df_bg= pd.concat([polygon_block_groups, polygon_block_groups_0, polygon_block_groups_1, polygon_block_groups_2, polygon_block_groups_3, polygon_block_groups_4], axis=1)

In [ ]:
result_df_bg.shape

In [ ]:
result_df_bg.describe()

In [ ]:
df_describe = result_df_bg.describe()[:1]
df_describe

In [ ]:
def contains_zero(series):
    return (series == 0).any()

# Use apply to check each column
columns_with_zero = df_describe.apply(contains_zero)
true_columns = columns_with_zero[columns_with_zero].index.tolist()
true_columns

In [ ]:
variable_and_names.head()

In [ ]:
df_subset = variable_and_names[variable_and_names['variables'].isin(true_columns)]

df_subset['names'] = 'Variable_' + df_subset['names']
df_subset

# test variables

In [ ]:
import requests

API_KEY = 'YOUR_API_KEY'  # Place your API key here
VARIABLE = 'B09001_003E'  # Example variable

data = {
    'county': [],
    'tract': [],
    'block group': []
}
geo_status = pd.DataFrame(data)
geo_status

In [ ]:
def check_variable_by_geography(variable, api_key):
    # Common geographic levels to check
    geographies = ['county', 'tract', 'block group']
    for geo in geographies:
        url = f"https://api.census.gov/data/2022/acs/acs5?get=NAME,{variable}&for={geo}:*"
        response = requests.get(url + f"&key={api_key}")
        if response.status_code == 200:
            geo_status[f'{geo}'] = 1
            # print(f"YES {variable} @ {geo}")
            # break
        else:
            geo_status[f'{geo}'] = 0
            # print(f"NO  {variable} @ {geo}")

In [ ]:
# Run the check
for i in range(len(true_columns)):
    check_variable_by_geography(true_columns[i], api_key)

In [ ]:
#FIX THIS TO RUN IN A DATAFRAME WHERE IT UPDATES THE ROWS ACCORDING TO THE NUMBER OF VARIABLES
geo_status

In [ ]:
DATASET = 'B09001'
YEAR = 2022

In [ ]:
# cgeo.geo_path_snake_specs(DATASET, YEAR)
help(cgeo)

# Plotting

In [ ]:
import matplotlib.pyplot as plt

# This is a census variable for median household income.
# See https://api.census.gov/data/2020/acs/acs5/variables/B19013_001E.html
MEDIAN_HOUSEHOLD_INCOME_VARIABLE = "B19013_001E"

gdf_state_bounds = reader.read_cb_shapefile("us", "state")
gdf_state_bounds = gdf_state_bounds[
    gdf_state_bounds["STATEFP"].isin(states.ALL_STATES_AND_DC)
]

reader = cem.ShapeReader(year=YEAR)

In [ ]:
plt.rcParams["figure.figsize"] = (18, 8)


def plot_map(
    gdf: gpd.GeoDataFrame,
    geo: str,
    *,
    gdf_bounds: Optional[gpd.GeoDataFrame] = None,
    bounds_color: str = "white",
    max_income: float = 200_000.0,
):
    if gdf_bounds is None:
        gdf_bounds = gdf

    ax = cem.plot_us(gdf_bounds, color="lightgray")

    ax = cem.plot_us(
        gdf,
        MEDIAN_HOUSEHOLD_INCOME_VARIABLE,
        cmap="autumn",
        legend=True,
        vmin=0.0,
        vmax=max_income,
        ax=ax,
    )

    ax = cem.plot_us_boundary(gdf_bounds, edgecolor=bounds_color, linewidth=0.5, ax=ax)

    ax.set_title(f"{YEAR} Median Household Income by {geo.title()}")

    ax.axis("off")

In [ ]:
DATASET = 'acs/acs5'
YEAR = 2022
STATE = states.MA
VARIABLES = 'B09001_001E'

In [ ]:
gdf_bg = ced.download(
    DATASET,
    YEAR,
    VARIABLES,
    state=STATE,
    block_group="*",
    with_geometry=True,
    set_to_nan=cev.ALL_SPECIAL_VALUES,
)

In [ ]:
plt.rcParams["figure.figsize"] = (8, 8)

plot_map(
    gdf_bg,
    f"block group in {states.NAMES_FROM_IDS[STATE]}",
    gdf_bounds=gdf_state_bounds[gdf_state_bounds["STATEFP"] == STATE],
    bounds_color="black",
)